In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error

data = pd.read_csv("/content/DataJobID-2745450_2745450_Ekspor20002023.csv", encoding="ISO-8859-1")
data = data[['ProductCode', 'PartnerName', 'Year', 'TradeValue in 1000 USD']]

filtered_data = data.groupby(['ProductCode', 'PartnerName']).filter(lambda x: len(x) >= 5)

def forecast_arima(series, steps=7):
    series = series.sort_index()
    model = ARIMA(series, order=(1, 1, 1))
    fitted_model = model.fit()
    return fitted_model.forecast(steps=steps)

def forecast_lstm(series, steps=7):
    scaler = MinMaxScaler(feature_range=(0, 1))
    series_scaled = scaler.fit_transform(series.values.reshape(-1, 1))

    X, y = [], []
    for i in range(len(series_scaled) - 1):
        X.append(series_scaled[i:i+1])
        y.append(series_scaled[i+1])
    X, y = np.array(X), np.array(y)

    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(X.shape[1], 1)),
        LSTM(50),
        Dense(1)
    ])
    model.compile(optimizer=Adam(0.01), loss='mse')
    model.fit(X, y, epochs=50, batch_size=1, verbose=0)

    forecast = []
    last_sequence = series_scaled[-1].reshape(1, 1, 1)
    for _ in range(steps):
        pred = model.predict(last_sequence)
        forecast.append(pred[0][0])
        last_sequence = np.roll(last_sequence, -1)
        last_sequence[0, 0, -1] = pred

    return scaler.inverse_transform(np.array(forecast).reshape(-1, 1)).flatten()

results = []

for (product_code, partner_name), subset in filtered_data.groupby(['ProductCode', 'PartnerName']):
    subset = subset.set_index('Year')
    series = subset['TradeValue in 1000 USD']

    train = series[:-1]
    test = series[-1:]

    try:
        arima_forecast = forecast_arima(train, steps=len(test))
        arima_mae = mean_absolute_error(test, arima_forecast)
        arima_rmse = mean_squared_error(test, arima_forecast, squared=False)

        lstm_forecast = forecast_lstm(train, steps=len(test))
        lstm_mae = mean_absolute_error(test, lstm_forecast)
        lstm_rmse = mean_squared_error(test, lstm_forecast, squared=False)

        results.append({
            'ProductCode': product_code,
            'PartnerName': partner_name,
            'ARIMA_MAE': arima_mae,
            'ARIMA_RMSE': arima_rmse,
            'LSTM_MAE': lstm_mae,
            'LSTM_RMSE': lstm_rmse
        })

    except Exception as e:
        print(f"Error processing ProductCode {product_code} and PartnerName {partner_name}: {e}")

results_df = pd.DataFrame(results)
print(results_df)

In [ ]:
from prophet import Prophet
data = pd.read_csv("/content/new.csv", encoding="ISO-8859-1")
data = data[['ProductCode', 'PartnerName', 'Year', 'TradeValue in 1000 USD']]
filtered_data = data.groupby(['ProductCode', 'PartnerName']).filter(lambda x: len(x) >= 5)

def tune_prophet(series, steps=7):
    prophet_df = series.reset_index()
    prophet_df.columns = ['ds', 'y']
    prophet_df['ds'] = pd.to_datetime(prophet_df['ds'], format='%Y')

    param_grid = {
        'seasonality_mode': ['additive', 'multiplicative'],
        'changepoint_prior_scale': [0.01, 0.1, 0.5],
        'seasonality_prior_scale': [1.0, 5.0, 10.0]
    }

    best_rmse = float('inf')
    best_forecast = None
    best_params = {}

    for seasonality_mode in param_grid['seasonality_mode']:
        for changepoint_prior_scale in param_grid['changepoint_prior_scale']:
            for seasonality_prior_scale in param_grid['seasonality_prior_scale']:

                model = Prophet(
                    yearly_seasonality=True,
                    seasonality_mode=seasonality_mode,
                    changepoint_prior_scale=changepoint_prior_scale,
                    seasonality_prior_scale=seasonality_prior_scale,
                    daily_seasonality=False,
                    weekly_seasonality=False
                )
                model.fit(prophet_df)

                future = model.make_future_dataframe(periods=steps, freq='Y')
                forecast = model.predict(future)
                forecasted_values = forecast[['ds', 'yhat']].tail(steps).set_index('ds')['yhat']

                rmse = mean_squared_error(series[-steps:], forecasted_values[:steps], squared=False)

                if rmse < best_rmse:
                    best_rmse = rmse
                    best_forecast = forecasted_values
                    best_params = {
                        'seasonality_mode': seasonality_mode,
                        'changepoint_prior_scale': changepoint_prior_scale,
                        'seasonality_prior_scale': seasonality_prior_scale
                    }

    return best_forecast, best_rmse, best_params

results = []

for (product_code, partner_name), subset in filtered_data.groupby(['ProductCode', 'PartnerName']):
    subset = subset.set_index('Year')
    series = subset['TradeValue in 1000 USD']

    try:
        extended_forecast, best_rmse, best_params = tune_prophet(series, steps=7)

        for year, forecast_value in zip(range(2024, 2031), extended_forecast):
            results.append({
                'ProductCode': product_code,
                'PartnerName': partner_name,
                'Year': year,
                'Prophet_Forecast': forecast_value,
                'Best_RMSE': best_rmse,
                **best_params
            })

    except Exception as e:
        print(f"Error processing ProductCode {product_code} and PartnerName {partner_name}: {e}")

results_df = pd.DataFrame(results)

results_df.to_csv("/content/prophet_tuned_forecast_2024_2030.csv", index=False)

print(results_df)